# 230129 - TorchVision Object Detection Tutorial

[Link](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html)

In [1]:
!pip install torchvision
!pip install pycocotools

/bin/bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by /bin/bash)

         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by bash)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
/bin/bash: /opt/anaconda/envs/Python3/l

## 0. Background: Implementing with a custom dataset
- Dataset should inherit from `torch.utils.data.Dataset`
- Should implement `__len__` and `__getitem__`
- `__getitem__` should return
    - `boxes`, `FloatTensor[N, 4]`, coordinates of `N` bounding boxes `[x0, y0, x1, y1]`.
    - `labels`, `Int64Tensor[N]`, label for each bounding box, `0` representing background class.
    - `image_id`, `Int64Tensor[1]`, unique ID for each image in the dataset.
    - `area`, `Tensor[N]`, the area of each bounding box.
    - `iscrowd`, `UInt8Tensor[N]`, instances with `iscrowd=True` are ignored during evaluation.
    - (optionally) `masks` (`UInt8Tensor[N, H, W]`): The segmentation masks for each of the objects
    - (optionally) `keypoints` (`FloatTensor[N, K, 3]`): For each one of the N objects, it contains the K keypoints in `[x, y, visibility]` format, defining the object. `visibility=0` means that the keypoint is not visible. Note that for data augmentation, the notion of flipping a keypoint is dependent on the data representation, and you should probably adapt references/detection/transforms.py for your new keypoint representation.
    
`pycocotools` is a required dependency for evaluation.

## 1. Define Dataset

[Download this with `wget` and extract it with `unzip`](https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip).

Then write a `torch.utils.data.Dataset` for it.

In [2]:
import os
import numpy as np
import torch
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

## 2. Create Model

The model we'll be using is Mask-RCNN, which is like Faster-RCNN except it also outputs a segmentation mask.

![](https://pytorch.org/tutorials/_static/img/tv_tutorial/tv_image03.png)

The next two sections outline two alternative approaches to training a model: using a pretrained Faster-RCNN model, or replacing the head of an image classifier.

## 2.1 Preparing a Pretrained Model for Finetuning

In [3]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# Replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background

# Get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

## 2.2 Modifying the Model to Add a Different Backbone

An alternative approach is to create a `FasterRCNN` model with a different backbone.

In [4]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# Load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features

# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# Let's make the RPN (region proposal network) generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# Let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

# Put the pieces together inside a FasterRCNN model
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

## 2.3 We'll be using the approach described in Section 2.1

In [5]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # Load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # Get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # Replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # Now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

## 2.4 Putting Everything Together

You need the helper functions in `references/detection` [here](https://github.com/pytorch/vision). Clone the repo and copy the `detection` dir to the same directory as this notebook.

In [6]:
import detection.transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

## 2.5 Testing `forward()`

In [7]:
from detection.utils import collate_fn

# Initialize model.
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# Initialize dataset.
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))

# Create data loader from dataset.
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
     collate_fn=collate_fn
)

In [8]:
# What the model expects during training...

images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections

In [9]:
# ... And during inference.

model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

In [10]:
# Create main function which performs the training and validation.

from detection.engine import train_one_epoch, evaluate


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 2
    # use our dataset and defined transformations
    dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
    dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))

    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices[:-50])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=collate_fn)

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        
        # update the learning rate
        lr_scheduler.step()
        
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

In [ ]:
main()

Epoch: [0]  [ 0/60]  eta: 0:16:46  lr: 0.000090  loss: 5.1315 (5.1315)  loss_classifier: 0.6482 (0.6482)  loss_box_reg: 0.2594 (0.2594)  loss_mask: 4.2190 (4.2190)  loss_objectness: 0.0013 (0.0013)  loss_rpn_box_reg: 0.0035 (0.0035)  time: 16.7823  data: 0.3639
Epoch: [0]  [10/60]  eta: 0:12:50  lr: 0.000936  loss: 1.7115 (2.4789)  loss_classifier: 0.4585 (0.4400)  loss_box_reg: 0.2831 (0.3230)  loss_mask: 0.7977 (1.6905)  loss_objectness: 0.0154 (0.0188)  loss_rpn_box_reg: 0.0061 (0.0066)  time: 15.4197  data: 0.0391
Epoch: [0]  [30/60]  eta: 0:07:49  lr: 0.002629  loss: 0.6410 (1.2954)  loss_classifier: 0.1108 (0.2378)  loss_box_reg: 0.2498 (0.2813)  loss_mask: 0.2284 (0.7579)  loss_objectness: 0.0075 (0.0120)  loss_rpn_box_reg: 0.0044 (0.0064)  time: 15.7608  data: 0.0086
Epoch: [0]  [40/60]  eta: 0:05:14  lr: 0.003476  loss: 0.4398 (1.0892)  loss_classifier: 0.0585 (0.1963)  loss_box_reg: 0.2245 (0.2619)  loss_mask: 0.1914 (0.6138)  loss_objectness: 0.0070 (0.0107)  loss_rpn_box_re